In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
import json

from pprint import pprint

In [3]:
val_df = pd.read_json('data/val.json', orient = 'index')
val_df.reset_index(inplace=True)
val_df = val_df.rename(columns = {'index': 'user_id'})

In [4]:
val_df

,user_id,target,features
0,user_127756,female,"{'orders': [{'site-id': 407, 'orders': [{'crea..."
1,user_127757,male,"{'orders': [{'site-id': 16, 'orders': [{'creat..."
2,user_127758,female,"{'orders': [{'site-id': 149, 'orders': [{'crea..."
3,user_127759,female,"{'orders': [{'site-id': 93, 'orders': [{'creat..."
4,user_127760,female,"{'visits': [{'site-id': 3, 'first-seen': 16836..."
...,...,...,...
27442,user_155198,female,"{'orders': [{'site-id': 7, 'orders': [{'create..."
27443,user_155199,male,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
27444,user_155200,female,"{'orders': [{'site-id': 34, 'orders': [{'creat..."
27445,user_155201,male,"{'orders': [{'site-id': 218, 'orders': [{'crea..."


In [23]:
val_df_fixed = val_df.join(pd.json_normalize(val_df['features'])).drop('features', axis='columns')
val_df_fixed

,user_id,target,orders,visits,site-meta,exchange-sessions,last-visits-in-categories
0,user_127756,female,"[{'site-id': 407, 'orders': [{'created-at': 16...","[{'site-id': 3, 'first-seen': 1697125590, 'las...",[{'site-id': 407}],"[{'landed-at': 1697132982, 'sites': [169, 214,...","[{'category': 'other', 'last-visit-at': 169713..."
1,user_127757,male,"[{'site-id': 16, 'orders': [{'created-at': 163...","[{'site-id': 3, 'first-seen': 1698423484, 'las...","[{'site-id': 16, 'recency': 1, 'frequency': 5,...","[{'landed-at': 1693508972, 'sites': [2, 169, 7...","[{'category': 'electronics', 'last-visit-at': ..."
2,user_127758,female,"[{'site-id': 149, 'orders': [{'created-at': 16...","[{'site-id': 391, 'first-seen': 1685615371, 'l...","[{'site-id': 391}, {'site-id': 42}, {'site-id'...",NaN,"[{'category': 'sport', 'last-visit-at': 165060..."
3,user_127759,female,"[{'site-id': 93, 'orders': [{'created-at': 162...","[{'site-id': 495, 'first-seen': 1687325784, 'l...","[{'site-id': 93, 'recency': 5, 'frequency': 2,...","[{'landed-at': 1697791749, 'sites': [373, 169,...","[{'category': 'furniture', 'last-visit-at': 16..."
4,user_127760,female,NaN,"[{'site-id': 3, 'first-seen': 1683626530, 'las...","[{'site-id': 46}, {'site-id': 29}, {'site-id':...",NaN,"[{'category': 'other', 'last-visit-at': 169765..."
...,...,...,...,...,...,...,...
27442,user_155198,female,"[{'site-id': 7, 'orders': [{'created-at': 1625...","[{'site-id': 3, 'first-seen': 1688903751, 'las...","[{'site-id': 49, 'recency': 1, 'frequency': 1,...","[{'landed-at': 1650900543, 'sites': [71, 326, ...","[{'category': 'other', 'last-visit-at': 169699..."
27443,user_155199,male,"[{'site-id': 21, 'orders': [{'created-at': 169...","[{'site-id': 21, 'first-seen': 1697887543, 'la...","[{'site-id': 21}, {'site-id': 17}]",NaN,"[{'category': 'hypermarket', 'last-visit-at': ..."
27444,user_155200,female,"[{'site-id': 34, 'orders': [{'created-at': 158...","[{'site-id': 22, 'first-seen': 1686412090, 'la...","[{'site-id': 34, 'recency': 1, 'frequency': 1,...",NaN,"[{'category': 'furniture', 'last-visit-at': 16..."
27445,user_155201,male,"[{'site-id': 218, 'orders': [{'created-at': 16...","[{'site-id': 555, 'first-seen': 1696063525, 'l...","[{'site-id': 218, 'recency': 1, 'frequency': 1...",NaN,"[{'category': 'other', 'last-visit-at': 169911..."


In [62]:
def explode_column(df, level_labels):
    for level in level_labels:
        df = df.explode(level)
        df = df.rename(columns = {level: level+'_old'})
        df = df.join(pd.json_normalize(df[level+'_old'])).drop(level+'_old', axis='columns')
    return df

# orders

In [59]:
val_orders = val_df_fixed[['user_id', 'target', 'orders']].copy()
val_orders

,user_id,target,orders
0,user_127756,female,"[{'site-id': 407, 'orders': [{'created-at': 16..."
1,user_127757,male,"[{'site-id': 16, 'orders': [{'created-at': 163..."
2,user_127758,female,"[{'site-id': 149, 'orders': [{'created-at': 16..."
3,user_127759,female,"[{'site-id': 93, 'orders': [{'created-at': 162..."
4,user_127760,female,NaN
...,...,...,...
27442,user_155198,female,"[{'site-id': 7, 'orders': [{'created-at': 1625..."
27443,user_155199,male,"[{'site-id': 21, 'orders': [{'created-at': 169..."
27444,user_155200,female,"[{'site-id': 34, 'orders': [{'created-at': 158..."
27445,user_155201,male,"[{'site-id': 218, 'orders': [{'created-at': 16..."


In [63]:
levels_labels = ['orders', 'orders', 'items']
val_orders_flat = explode_column(val_orders, levels_labels)
val_orders_flat

,user_id,target,site-id,created-at,id,count,general-category-path,brand-id
0,user_127756,female,407.0,1.697126e+09,item_676374,NaN,NaN,NaN
1,user_127757,male,16.0,1.636574e+09,item_3780263,2.0,"[90578, 90574, 198118]",1237.0
1,user_127757,male,16.0,1.636574e+09,item_3780263,2.0,"[90578, 90574, 198118]",1237.0
1,user_127757,male,16.0,1.636574e+09,item_3780263,2.0,"[90578, 90574, 198118]",1237.0
1,user_127757,male,16.0,1.636574e+09,item_3780263,2.0,"[90578, 90574, 198118]",1237.0
...,...,...,...,...,...,...,...,...
27446,user_155202,female,10.0,1.605151e+09,item_260022,1.0,"[90555, 944108, 198119]",2.0
27446,user_155202,female,10.0,1.605151e+09,item_260022,1.0,"[90555, 944108, 198119]",2.0
27446,user_155202,female,10.0,1.605151e+09,item_260022,1.0,"[90555, 944108, 198119]",2.0
27446,user_155202,female,10.0,1.605151e+09,item_260022,1.0,"[90555, 944108, 198119]",2.0


# visits

In [66]:
val_visits = val_df_fixed[['user_id', 'target', 'visits']].copy()
val_visits

,user_id,target,visits
0,user_127756,female,"[{'site-id': 3, 'first-seen': 1697125590, 'las..."
1,user_127757,male,"[{'site-id': 3, 'first-seen': 1698423484, 'las..."
2,user_127758,female,"[{'site-id': 391, 'first-seen': 1685615371, 'l..."
3,user_127759,female,"[{'site-id': 495, 'first-seen': 1687325784, 'l..."
4,user_127760,female,"[{'site-id': 3, 'first-seen': 1683626530, 'las..."
...,...,...,...
27442,user_155198,female,"[{'site-id': 3, 'first-seen': 1688903751, 'las..."
27443,user_155199,male,"[{'site-id': 21, 'first-seen': 1697887543, 'la..."
27444,user_155200,female,"[{'site-id': 22, 'first-seen': 1686412090, 'la..."
27445,user_155201,male,"[{'site-id': 555, 'first-seen': 1696063525, 'l..."


In [68]:
levels_labels = ['visits', 'visits']
val_visits_flat = explode_column(val_visits, levels_labels)
val_visits_flat

,user_id,target,site-id,first-seen,last-seen,visited-at,session-duration,pages-count,visited-items,visited-general-categories,visited-universal-brands
0,user_127756,female,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,NaN,NaN,NaN
0,user_127756,female,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,NaN,NaN,NaN
0,user_127756,female,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,NaN,NaN,NaN
0,user_127756,female,3.0,1.697126e+09,1.697133e+09,1.697126e+09,0.0,1.0,NaN,NaN,NaN
1,user_127757,male,407.0,1.697125e+09,1.699115e+09,1.697133e+09,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
27445,user_155201,male,776.0,1.686662e+09,1.694961e+09,1.676268e+09,0.0,1.0,NaN,NaN,NaN
27445,user_155201,male,776.0,1.686662e+09,1.694961e+09,1.676268e+09,0.0,1.0,NaN,NaN,NaN
27445,user_155201,male,776.0,1.686662e+09,1.694961e+09,1.676268e+09,0.0,1.0,NaN,NaN,NaN
27445,user_155201,male,776.0,1.686662e+09,1.694961e+09,1.676268e+09,0.0,1.0,NaN,NaN,NaN


# site-meta

In [69]:
val_site_meta = val_df_fixed[['user_id', 'target', 'site-meta']].copy()
val_site_meta

,user_id,target,site-meta
0,user_127756,female,[{'site-id': 407}]
1,user_127757,male,"[{'site-id': 16, 'recency': 1, 'frequency': 5,..."
2,user_127758,female,"[{'site-id': 391}, {'site-id': 42}, {'site-id'..."
3,user_127759,female,"[{'site-id': 93, 'recency': 5, 'frequency': 2,..."
4,user_127760,female,"[{'site-id': 46}, {'site-id': 29}, {'site-id':..."
...,...,...,...
27442,user_155198,female,"[{'site-id': 49, 'recency': 1, 'frequency': 1,..."
27443,user_155199,male,"[{'site-id': 21}, {'site-id': 17}]"
27444,user_155200,female,"[{'site-id': 34, 'recency': 1, 'frequency': 1,..."
27445,user_155201,male,"[{'site-id': 218, 'recency': 1, 'frequency': 1..."


In [70]:
levels_labels = ['site-meta']
val_site_meta_flat = explode_column(val_site_meta, levels_labels)
val_site_meta_flat

,user_id,target,site-id,recency,frequency,monetary
0,user_127756,female,407,NaN,NaN,NaN
1,user_127757,male,16,1.0,5.0,4.0
1,user_127757,male,16,1.0,5.0,4.0
1,user_127757,male,16,1.0,5.0,4.0
1,user_127757,male,16,1.0,5.0,4.0
...,...,...,...,...,...,...
27446,user_155202,female,16,NaN,NaN,NaN
27446,user_155202,female,16,NaN,NaN,NaN
27446,user_155202,female,16,NaN,NaN,NaN
27446,user_155202,female,16,NaN,NaN,NaN


# exchange-sessions

In [72]:
val_exch_sess = val_df_fixed[['user_id', 'target', 'exchange-sessions']].copy()
val_exch_sess

,user_id,target,exchange-sessions
0,user_127756,female,"[{'landed-at': 1697132982, 'sites': [169, 214,..."
1,user_127757,male,"[{'landed-at': 1693508972, 'sites': [2, 169, 7..."
2,user_127758,female,NaN
3,user_127759,female,"[{'landed-at': 1697791749, 'sites': [373, 169,..."
4,user_127760,female,NaN
...,...,...,...
27442,user_155198,female,"[{'landed-at': 1650900543, 'sites': [71, 326, ..."
27443,user_155199,male,NaN
27444,user_155200,female,NaN
27445,user_155201,male,NaN


In [75]:
levels_labels = ['exchange-sessions', 'clicks']
val_exch_sess_flat = explode_column(val_exch_sess, levels_labels)
val_exch_sess_flat

,user_id,target,landed-at,sites,accepted-site-id,accepted-at,clicked-at,site-id
0,user_127756,female,1.697133e+09,"[169, 214, 232, 244, 405, 229, 83, 231, 2, 21,...",262.0,1.697133e+09,NaN,NaN
0,user_127756,female,1.697133e+09,"[169, 214, 232, 244, 405, 229, 83, 231, 2, 21,...",262.0,1.697133e+09,NaN,NaN
1,user_127757,male,1.697126e+09,"[169, 214, 837, 476, 216, 215, 2, 83, 21, 244,...",169.0,1.697126e+09,NaN,NaN
2,user_127758,female,1.693509e+09,"[2, 169, 71, 236, 218, 21, 240, 215, 239, 226,...",42.0,1.693509e+09,NaN,NaN
3,user_127759,female,NaN,NaN,NaN,NaN,1.693509e+09,42.0
...,...,...,...,...,...,...,...,...
27446,user_155202,female,1.522666e+09,"[196, 203, 93, 58, 44, 147, 179, 53, 346, 86, ...",196.0,1.522666e+09,NaN,NaN
27446,user_155202,female,1.522666e+09,"[196, 203, 93, 58, 44, 147, 179, 53, 346, 86, ...",196.0,1.522666e+09,NaN,NaN
27446,user_155202,female,1.522666e+09,"[196, 203, 93, 58, 44, 147, 179, 53, 346, 86, ...",196.0,1.522666e+09,NaN,NaN
27446,user_155202,female,1.522666e+09,"[196, 203, 93, 58, 44, 147, 179, 53, 346, 86, ...",196.0,1.522666e+09,NaN,NaN


# last-visits-in-categories

In [64]:
val_last_visits = val_df_fixed[['user_id', 'target', 'last-visits-in-categories']].copy()
val_last_visits

,user_id,target,last-visits-in-categories
0,user_127756,female,"[{'category': 'other', 'last-visit-at': 169713..."
1,user_127757,male,"[{'category': 'electronics', 'last-visit-at': ..."
2,user_127758,female,"[{'category': 'sport', 'last-visit-at': 165060..."
3,user_127759,female,"[{'category': 'furniture', 'last-visit-at': 16..."
4,user_127760,female,"[{'category': 'other', 'last-visit-at': 169765..."
...,...,...,...
27442,user_155198,female,"[{'category': 'other', 'last-visit-at': 169699..."
27443,user_155199,male,"[{'category': 'hypermarket', 'last-visit-at': ..."
27444,user_155200,female,"[{'category': 'furniture', 'last-visit-at': 16..."
27445,user_155201,male,"[{'category': 'other', 'last-visit-at': 169911..."


In [65]:
levels_labels = ['last-visits-in-categories']
val_last_visits_flat = explode_column(val_last_visits, levels_labels)
val_last_visits_flat

,user_id,target,category,last-visit-at
0,user_127756,female,other,1.697133e+09
0,user_127756,female,other,1.697133e+09
1,user_127757,male,education,1.699115e+09
1,user_127757,male,education,1.699115e+09
1,user_127757,male,education,1.699115e+09
...,...,...,...,...
27445,user_155201,male,fashion,1.682684e+09
27445,user_155201,male,fashion,1.682684e+09
27445,user_155201,male,fashion,1.682684e+09
27445,user_155201,male,fashion,1.682684e+09
